### 서브그래프 사용해보기
왜 서브그래프가 필요한가? 복잡한 워크플로우를 모듈화를 통해 분리하거나, 재사용하는 그래프가 필요한 경우!

In [25]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict
import random

In [26]:
# 상태 정의
class State(TypedDict):
    user_input: int
    computer_choice: int
    result: str

In [27]:
builder_A = StateGraph(State)

def nodeA(state: State) -> State:
    return {"result": f"You win! Your input ({state['user_input']}) >= Computer ({state['computer_choice']})"}

builder_A.add_node("nodeA", nodeA)
builder_A.add_edge(START, "nodeA")
builder_A.add_edge("nodeA", END)

graphA = builder_A.compile()

In [28]:
builder_B = StateGraph(State)

def nodeB(state: State) -> State:
    return {"result": f"You lose... Your input ({state['user_input']}) < Computer ({state['computer_choice']})"}

builder_B.add_node("nodeB", nodeB)
builder_B.add_edge(START, "nodeB")
builder_B.add_edge("nodeB", END)
graphB = builder_B.compile()

In [29]:
# 메인 그래프 만들기
main_builder = StateGraph(State)

def judge_input(state: State) -> str:
    if state["user_input"] >= state["computer_choice"]:
        return "run_graph_a"
    else:
        return "run_graph_b"
    
main_builder.add_node("run_graph_a", graphA)
main_builder.add_node("run_graph_b", graphB)

# Conditional routing from START
main_builder.add_conditional_edges(
    START,
    judge_input,
    {
        "run_graph_a": "run_graph_a",
        "run_graph_b": "run_graph_b"
    }
)

main_builder.add_edge("run_graph_a", END)
main_builder.add_edge("run_graph_b", END)

graph = main_builder.compile()

In [30]:
user = int(input("Enter a number (1~10): "))
computer = random.randint(1, 10)

result = graph.invoke({"user_input": user, "computer_choice": computer})
print("Final State:", result)

Final State: {'user_input': 5, 'computer_choice': 7, 'result': 'You lose... Your input (5) < Computer (7)'}
